In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# A Parametric Representation

The parametric form of a curve is nothing more than ***a machine for making points.*** It arises through ***the application of a function that maps an interval of real numbers to a set of points in space***. 


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P01.jpg" style="width: 600px; display: inline;">

By mathematical convention, the function (a vector function) is denoted as $c$ and the set of points by $c(t)$ for $t$ varying in the interval $[a,b]$. 

Taken together, these three elements – ***a function, an interval, and a resulting set of points*** – define what we call a curve.

There is ***a natural orientation*** to the curve since we start from one endpoint $c(a)$ and proceed along the curve in the direction of increasing $t$ until reaching the other endpoint $c(b)$. 

If we were speaking more formally, ***the function $c$ parameterizes a curve $C$ *** . 

If $C$ is a curve in the plane, our function would only need to produce points with $x$ and $y$ coordinates, and we could describe our curve mathematically with coordinate functions: 

$c(t) = (x(t), y(t)) $ 

A curve in space needs to define three coordinates of each resulting point, and would be written as:

$c(t) = (x(t), y(t), z(t)) $ 

## Mathematical Monsters Redux

Reflecting back on our Mathematical Monsters example will shed some light on these ideas.

In [ ]:
"""
Mathematical Monsters Template
Plots a set of points in space by hybridizing formulas for known curves
"""
ival = Interval(a,b)
pts = []
for t in ival / count:
    x = some_math
    y = some_more_math
    pts.append(Point(x,y))


In [ ]:
# an evaluation function
def evaluate_monster_at(t):
    x = some_math
    y = some_more_math
    return Point(x,y)

In [ ]:
# a set of points
pts = []
# a loop that iterates across a given interval
for t in Interval(a,b) / count: 
    pts.append( evaluate_monster_at(t) )

We may notice some similarities between this code and the mathematical description of parametric curves presented above. 

* The function `evaluate_monster_at()` may be understood as a parameterization function which describes a smooth curve.
* The interval called for in the definition of a parametric curve is expressed in code as `Interval(a,b)`.

Practically, we can only call upon the evaluation function so many times. In order to define a curve in our own scripts or in software, we must also define the size of this sample set, a quantity which determines the effective ***tolerance*** of our curve.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P03.jpg" style="width: 200px; display: inline;">

A comment regarding notation.

We will often describe curves using the following modified (and unorthodox) mathematical notation for describing curves:

\begin{align}
x = \ldots \\
y = \ldots \\
z = \ldots \\
t\colon a\rightarrow b
\end{align}

## Curve Objects in Decod.es

How can we go about constructing a Curve class that formalizes the constructions and operations we may want to perform on curves? 

We might begin by defining the three basic elements: 
* a function
* an interval
* a resulting set of points.

Since functions are first-class objects in Python, our Curve class can easily store the ***function*** and ***Interval*** as members, and require them to be provided to the Curve constructor. 

The last element, the ***resulting set of points***, is less straightforward. Here we offer a method for constructing Points at whatever position along our Curve that we wish, an operation referred to as ***evaluating the Curve***. As this is more of an operation than data to be stored, we can structure this as a method.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D72 Curve Large.jpg" style="width: 800px; display: inline;">

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Basic Curve Members*</th>
    </tr>
    <tr>
        <td style="width:20%">`crv._func`<br>`crv.func`</td>
        <td style="width:20%">Function</td>
        <td style="width:60%">A function that, given a single parameter `t`, returns a Point in space.</td>
    </tr>
    <tr>
        <td style="width:20%">`crv._dom`<br>`crv.domain`</td>
        <td style="width:20%">Interval</td>
        <td style="width:60%">An Interval of valid t-values to evaluate.</td>
    </tr>
    <tr>
        <td style="width:20%">`crv._tol`<br>`crv.tol`</td>
        <td style="width:20%">Float</td>
        <td style="width:60%">The tolerance of the curve expressed in domain space.</td>
    </tr>
</table>

Note that the three basic members (`crv._func`, `crv._dom`, and `crv._tol`) are private, but made accessible via similarly named properties (`crv.func`, `crv.domain`, `crv.tol`), and that in the constructor the tolerance parameter has been made optional.

In [ ]:
"""
Curve Initialization
A Curve is constructed of an evaluation function, a domain Interval, and a tolerance value.
"""
class Curve():
    def __init__(self, function, domain=Interval(0,1), tolerance=None):
        self.func = function
        self.domain = domain
        self.tol = tolerance

Evaluating the Curve is simple enough: given a parameter `t` that falls within the defined domain, we can simply call the stored function and return the resulting Point.

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Basic Curve Methods*</th>
    </tr>
    <tr>
        <td style="width:20%">`crv.eval(t)`</td>
        <td style="width:20%">Point</td>
        <td style="width:60%">Given a parameter `t`, returns a Point that falls on this Curve within the defined domain.</td>
    </tr>
    <tr>
        <td style="width:20%">`crv.deval(t)`</td>
        <td style="width:20%">Point</td>
        <td style="width:60%">Given a parameter `t`, returns a Point that falls on this Curve within a normalized domain.</td>
    </tr>
</table>

In [ ]:
"""
Normalized Evaluation
Evaluates this Curve and returns a Point. The argument t is a normalized float value (0-\>1) which 
will be remapped to the domain defined by this Curve.
"""
    def eval(self,t):
        return self.deval(Interval.remap(t,Interval(),self.domain))

In [ ]:
"""
Domain Evaluation
Evaluates this Curve and returns a Point. The argument t is a float value that falls within the 
defined domain of this Curve.
"""    
    def deval(self,t):
        pt = self.func(t)
        return pt

Given that we are working with a discrete approximation of a curve, it is useful to access a polyline representation that has been constructed out of sample points with resolution set by `crv.tol`. This approximation is a useful “surrogate” to a Curve and can be added as a property.

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Basic Curve Properties*</th>
    </tr>
    <tr>
        <td style="width:20%">`crv.surrogate`</td>
        <td style="width:20%">PLine</td>
        <td style="width:60%">Returns a Polyline representation of this Curve constructed of a sub-sampling of Points. The number of points in this Polyline is determined by dividing `crv.domain.delta` by `crv.tol`.</td>
    </tr>
</table>

In [ ]:
"""
Curve Surrogate
Returns a polyline representation of this Curve
"""
    @property
    def surrogate(self):
        res = int(math.ceil(self.domain.delta/self.tol))
        t_vals = self.domain.divide(res,True)
        return PLine([self.deval(t) for t in t_vals])

With this basic framework in place, we can draw a clear correlation mapping between the general notation we have been using to describe curves and the functional description using this Curve class. 

A curve in the plane can be expressed as:

\begin{align}
x = \ldots \\
y = \ldots \\
t\colon a\rightarrow b
\end{align}

Holds a direct mapping to code as:

In [ ]:
def func(t):
    x = ...
    y = ...
    return Point(x,y)

ival = Interval(a,b)
crv = Curve(func,ival)

### Visualizing Curves

While many of the geometric types we have discussed thus far have had
direct analogs in CAD, this does not hold true for the Decod.es Curve. This makes translation from a general description of curves via a Python function much less straightforward, as the representation of curves is so bound up in the code we use to define it.

Decod.es Curves can be approximately visualized through the built-in polyline representation `crv.surrogate`, or by manually plotting points at evenly spaced parameter values in the domain interval. 

***To see a Decod.es curve in CAD, we must plot it***.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P04.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Plotting a Curve
A Curve object may be visualized by accessing its surrogate, a PLine with a number of Points 
related to Curve.tol, or by successively evaluating the Curve using eval or deval. Each of the 
following bits of code are more-or-less equivalent.
"""
# plot via crv.surrogate
crv.surrogate

# plot via crv.eval()
pts = []
for t in Interval()/30: 
    pts.append(crv.eval(t))
	
# plot via crv.deval()
pts = []
for t in crv.domain/30: 
    pts.append(crv.deval(t))

# Features and Idiosyncrasies

Here we consider some features that naturally come with the territory of the parametric representation, and that may not immediately align with our assumptions about curves.

### Unequal Division

Equally-spaced points along a parameterized curve are not always 'natural'. 

Take, for example, the parabola:

\begin{align*}
x &= t\\
y &= t^2\\
t\colon 0&\rightarrow 2
\end{align*}

From the point of view of the parameterization, the most natural points to access are those corresponding to equally-spaced parameter values in the domain interval.

We can see that these points are not equally spaced along the curve.

The underlying representation of all but the most simple of curves exhibits this feature.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P02.jpg" style="width: 200px; display: inline;">

Uneven divsion is not usually felt when working in CAD environments, thanks to built-in commands such as `“Divide”` which distributes equally spaced points along a curve. 

At this lower level representation, however, it is important to understand that there is nothing automatic about this functionality.

### Non-Unique Parameterizations

***A curve can always be described by multiple parameterizations.***

The fact that a curve can have multiple parameterizations is one that holds tremendous conceptual and practical implications for design. Armed with this knowledge, we can construct a parameterization with points that we wish to easily access in mind.

We demonstrate this on the simplest of curves: the line and the circle.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P05.jpg" style="width: 600px; display: inline;">

In [ ]:
"""
A Strange Line Curve
Demonstrates a whole family of parametrizations for the line, each with Points spaced along it 
according to a power function
"""
pt_a, pt_b = Point(), Point(1,1)
n = 4

def func(t):
    x = pt_a.x + (pt_b.x - pt_a.x)*t**n
    y = pt_a.y + (pt_b.y - pt_a.y)*t**n
    return Point(x,y)

ival = Interval() 
tol = 1.0/100
strange_line = Curve(func,ival,tol)

pts = [strange_line.eval(t) for t in Interval()/100]
out.put(pts)

out.draw()
out.clear()

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P06.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
The "normal" parameterization of a circle, as seen in the upper diagram
"""
out = JupyterOut.origin_centered( 100.0 )

def eval_normal(t):
    x = math.cos(t)
    y = math.sin(t)
    return Point(x,y)

ival = Interval.twopi()
cir_normal = Curve(eval_normal,ival)

pts = [eval_normal(t) for t in ival/100]
out.put(pts)

out.draw()
out.clear()

In [ ]:
"""
The "goofy" parameterization of a circle, as seen in the lower diagram
"""
def eval_goofy(t):
    x = (1 - t**2)/(1 + t**2)
    y = 2*t/(1+t**2)
    return Point(x,y)

ival = Interval(-50,50) 
cir_goofy = Curve(eval_goofy,ival)

pts = [eval_goofy(t) for t in ival/100]
out.put(pts)

out.draw()
out.clear()

## Composing with Parametric Curves

The functional approach to constructing curves may initially strike a designer as an unintuitive one, as it is quite unlike that of constructing curves via control points.

Most modern CAD environments operate more intuitively. In contrast, directly accessing the parametric form requires first deciding on both the properties desired and the best means to describe these properties before anything can
be seen.

Rather than ***iteratively altering the shape of a curve until it looks right***, this functional approach allows us to ***iteratively alter a description of what the curve does until it acts right***.

As such, it is worth becoming comfortable with creating and manipulating geometry in this extraordinarily powerful way.